# 5.0 MNIST

In [1]:
from clustergrammer2 import net
df = {}

clustergrammer2 backend version 0.2.9


In [ ]:
import clustergrammer_groupby as cby
from copy import deepcopy
import random
random.seed(99)

In [ ]:
net.load_file('../data/big_data/MNIST_row_labels.txt')
df['mnist'] = net.export_df()
df['mnist'].shape

In [ ]:
cols = df['mnist'].columns.tolist()
new_cols = [(x, 'Digit: ' + x.split('-')[0]) for x in cols]
df['mnist-cat'] = deepcopy(df['mnist'])
df['mnist-cat'].columns = new_cols
print(new_cols[0])

### Make Train and Predict

In [ ]:
cols = df['mnist-cat'].columns.tolist()
random.shuffle(cols)
df['mnist-train'] = df['mnist-cat'][cols[:35000]]
df['mnist-pred'] = df['mnist-cat'][cols[35000:]]
print(df['mnist-train'].shape, df['mnist-pred'].shape)

### Make Z-scored Data

In [ ]:
net.load_df(df['mnist-train'])
net.normalize(axis='row', norm_type='zscore')
df['mnist-train-z'] = net.export_df()

In [ ]:
net.load_df(df['mnist-pred'])
net.normalize(axis='row', norm_type='zscore')
df['mnist-pred-z'] = net.export_df()

### Make Signatures

In [ ]:
pval_cutoff = 0.00001
num_top_dims = 50

In [ ]:
df['sig'], keep_genes_dict, df_gene_pval, fold_info = cby.generate_signatures(df['mnist-train'],
                                                                     'Digit', num_top_dims=num_top_dims)
df['sig'].shape

In [ ]:
df['sig-z'], keep_genes_dict, df_gene_pval, fold_info = cby.generate_signatures(df['mnist-train-z'],
                                                                     'Digit', num_top_dims=num_top_dims)
df['sig-z'].shape

In [ ]:
net.load_df(df['sig'])
net.widget()

In [ ]:
net.load_df(df['sig-z'])
net.widget()

# Predict Digit Type Using Signatures

### Raw Data

In [ ]:
# Predict
##################
df_pred_cat, df_sig_sim, y_info = cby.predict_cats_from_sigs(df['mnist-pred'], df['sig'], truth_level=1,
                                                                   predict_level='Pred Digit', unknown_thresh=0.0)

df_conf, population, ser_correct, fraction_correct = cby.confusion_matrix_and_correct_series(y_info)
print('Predict: ', fraction_correct)

df_conf, population, ser_correct, fraction_correct = cby.confusion_matrix_and_correct_series(y_info)
print('\nbroad cell type: ', fraction_correct, '\n')
print(ser_correct.sort_values(ascending=False))

In [ ]:
df_pred_cat.shape

In [ ]:
net.load_df(df_pred_cat)
net.random_sample(axis='col', num_samples=2500, random_state=100)
net.widget()

### Z-scored Data

In [ ]:
# Predict
##################
df_pred_cat, df_sig_sim, y_info = cby.predict_cats_from_sigs(df['mnist-pred-z'], df['sig-z'], truth_level=1,
                                                                   predict_level='Pred Digit', unknown_thresh=0.0)

df_conf, population, ser_correct, fraction_correct = cby.confusion_matrix_and_correct_series(y_info)
print('Predict: ', fraction_correct)

df_conf, population, ser_correct, fraction_correct = cby.confusion_matrix_and_correct_series(y_info)
print('\nbroad cell type: ', fraction_correct, '\n')
print(ser_correct.sort_values(ascending=False))